In [1]:
import os

In [2]:
import pandas as pd
import numpy as np

df = pd.read_csv(os.path.join(os.getcwd(),'Data','Amazon Search Terms_Search Terms_US.csv')).sample(n=1000,ignore_index=True)

In [3]:
from gensim.parsing.preprocessing import remove_stopwords
import string
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import webcolors

In [4]:
from sklearn.neighbors import NearestNeighbors

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [6]:
product_list = list()
for i in range(len(df['#1 Product Title'])):
    for j in range(1,4):
        product_list.append([df.loc[i,'Search Term'],df.loc[i,f'#{j} Product Title']])

In [7]:
df = pd.DataFrame(product_list,columns=['Search Term','Product Title'])

In [8]:
Y,X = df[['Product Title']], df[['Search Term']]

In [9]:
Y_raw,X_raw = Y.copy(),X.copy()

In [10]:
X_raw,Y_raw

(                 Search Term
 0                 anine bing
 1                 anine bing
 2                 anine bing
 3                    guylian
 4                    guylian
 ...                      ...
 2995  dinosaur costume adult
 2996  dinosaur costume adult
 2997        harness lingerie
 2998        harness lingerie
 2999        harness lingerie
 
 [3000 rows x 1 columns],
                                           Product Title
 0        ANINE BING Women's Arlo Desert Road Sweatshirt
 1                   ANINE BING Women's Vintage Bing Tee
 2        ANINE BING Women's Ramona Sweatshirt AB x to B
 3     GuyLian Belgian Chocolate Gift Box, Includes S...
 4     Guylian Belgian Chocolate Seashells (250g) - P...
 ...                                                 ...
 2995  Inflatable Dinosaur Costume Blow up Triceratop...
 2996  Decalare Adult/Kids Size Inflatable T-REX Dino...
 2997  LIVE4COOL Women Harness Cupless Cage Sexy Bra ...
 2998  Women's Punk Cut Out Harness Body F

In [11]:
def stemSentence(sentence):
    porter = PorterStemmer()
    token_words = word_tokenize(sentence)
    stem_sentence = [porter.stem(word) for word in token_words]
    return ' '.join(stem_sentence)
def Clean_Sequence(X=X,is_X = True):
    if is_X:
        products = pd.Series(list(X['Search Term']))
    else:
        products = pd.Series(list(X['Product Title']))
    
    products = [remove_stopwords(str(x))\
            .translate(str.maketrans('','',string.punctuation))\
            .translate(str.maketrans('','',string.digits))\
            for x in products]
    
    if is_X:
        X.loc[:,'Search Term'] = products
    else:
        X.loc[:,'Product Title'] = products
    
    if is_X:
        products = pd.Series(list(X['Search Term']))
    else:
        products = pd.Series(list(X['Product Title']))
    
    products = pd.Series([stemSentence(str(x)) for x in products])
    
    if is_X:
        X.loc[:,'Search Term'] = products
    else:
        X.loc[:,'Product Title'] = products
    
    colors = list(webcolors.CSS3_NAMES_TO_HEX)
    colors = [stemSentence(str(x)) for x in colors if x not in ('bisque','blanchedalmond','chocolate','honeydew','lime',
                                             'olive','orange','plum','salmon','tomato','wheat')]

    if is_X:
        products = pd.Series(list(X['Search Term']))
    else:
        products = pd.Series(list(X['Product Title']))
    
    products = [' '.join([x for x in str(string).split() if x not in colors]) for string in products]

    if is_X:
        X.loc[:,'Search Term'] = products
    else:
        X.loc[:,'Product Title'] = products

    return X

X = Clean_Sequence(X)
Y = Clean_Sequence(Y,is_X=False)

C:\Users\Ali\AppData\Local\Temp\ipykernel_22860\1346602226.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.loc[:,'Search Term'] = products
C:\Users\Ali\AppData\Local\Temp\ipykernel_22860\1346602226.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.loc[:,'Search Term'] = products
C:\Users\Ali\AppData\Local\Temp\ipykernel_22860\1346602226.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See t

In [12]:
X,Y

(                Search Term
 0                 anin bing
 1                 anin bing
 2                 anin bing
 3                   guylian
 4                   guylian
 ...                     ...
 2995  dinosaur costum adult
 2996  dinosaur costum adult
 2997            har lingeri
 2998            har lingeri
 2999            har lingeri
 
 [3000 rows x 1 columns],
                                           Product Title
 0           anin bing women arlo desert road sweatshirt
 1                       anin bing women vintag bing tee
 2              anin bing women ramona sweatshirt ab x b
 3     guylian belgian chocol gift box includ silki s...
 4                         guylian belgian chocol g pack
 ...                                                 ...
 2995  inflat dinosaur costum blow triceratop costum ...
 2996  decalar adultkid size inflat trex dinosaur cos...
 2997  livecool women har cupless cage sexi bra panti...
 2998  women punk cut out har bodi full strappi linge.

In [13]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_productid = tfidf_vectorizer.fit_transform((Y['Product Title']))
#Y_verctorized = pd.concat([Y,pd.DataFrame(Y_verctorized.toarray(),
#                            columns=Y_verctorized.get_feature_names_out())],axis=1)

In [14]:
def Search_system():
    Search_String = input('What to search? :')
    index = np.where(X_raw['Search Term'] == Search_String)[0][0]
    Search_q = X_raw.iloc[[index]]
    Search_q = Clean_Sequence(Search_q)
    Search_tfidf = tfidf_vectorizer.transform(Search_q['Search Term'])
    return [Search_tfidf,Search_String]

In [15]:
def get_recommendation(top, df_all, scores, Search_String):
    recommendation = pd.DataFrame(columns = ['Search Term',  'Results', 'score'])
    count = 0
    for i in top:
        recommendation.at[count, 'Search Term'] = Search_String
        recommendation.at[count, 'Results'] = df_all['Product Title'][i]
        recommendation.at[count, 'score'] =  scores[count]
        count += 1
    return recommendation.loc[:4,]

In [16]:
KNN = NearestNeighbors(n_neighbors = 11)
KNN.fit(tfidf_productid)
Searched = Search_system()
Search_tfidf,Search_string = Searched[0],Searched[1]
NNs = KNN.kneighbors(Search_tfidf, return_distance=True)
top = NNs[1][0][1:]
index_score = NNs[0][0][1:]
get_recommendation(top, Y_raw, index_score, Search_string)

What to search? :anine bing


C:\Users\Ali\AppData\Local\Temp\ipykernel_22860\1346602226.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.loc[:,'Search Term'] = products
C:\Users\Ali\AppData\Local\Temp\ipykernel_22860\1346602226.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.loc[:,'Search Term'] = products
C:\Users\Ali\AppData\Local\Temp\ipykernel_22860\1346602226.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See t

,Search Term,Results,score
0,anine bing,ANINE BING Women's Ramona Sweatshirt AB x to B,0.889665
1,anine bing,ANINE BING Women's Arlo Desert Road Sweatshirt,0.932085
2,anine bing,Aqua,1.0
3,anine bing,GIHOO 127PCS Olive Green Balloon Garland Arch ...,1.414214
4,anine bing,gmani I Fully Vaccinated Face Mask Washable Re...,1.414214
